- Факторы которые влияют на энергопотребление? - почему?
- найти факторы и сделать рейтинг факторов
- Один дом, один файл.
- PDF и код юпитера
- К репозиторию файл README


### Инициализация и загрузка данных

In [1]:
import pandas as pd

# Относительный путь к папке где лежать файлы
smart_meters_folder_path = 'smart-meters-in-london/'

# Получение полного пути файла
file_acorn_details = '{}acorn_details.csv'.format(smart_meters_folder_path)
file_uk_bank_holidays = '{}uk_bank_holidays.csv'.format(smart_meters_folder_path)
file_weather_daily_darksky = '{}weather_daily_darksky.csv'.format(smart_meters_folder_path)
file_weather_hourly_darksky = '{}weather_hourly_darksky.csv'.format(smart_meters_folder_path)
file_informations_households = '{}informations_households.csv'.format(smart_meters_folder_path)
file_daily_dataset = '{}daily_dataset.csv/daily_dataset.csv'.format(smart_meters_folder_path)

# Загрузка файлов в Pandas
df_acorn_details = pd.read_csv(file_acorn_details, encoding='ANSI')
df_uk_bank_holidays = pd.read_csv(file_uk_bank_holidays, encoding='ANSI')
df_weather_daily_darksky = pd.read_csv(file_weather_daily_darksky, encoding='ANSI')
df_weather_hourly_darksky = pd.read_csv(file_weather_hourly_darksky, encoding='ANSI')
df_informations_households = pd.read_csv(file_informations_households, encoding='ANSI')
df_daily_dataset = pd.read_csv(file_daily_dataset, encoding='ANSI')

In [ ]:
df_daily_dataset.head()

In [ ]:
df_informations_households.head()

### Тип счетчика в зависимости от класса ACORN
Хочу получить процентное соотношение ToU/Std в зависимости от класса

In [4]:
altered_df_informations_households = df_informations_households.loc[:][:]

In [5]:
def count_ToU(row):
    if row['stdorToU'] == 'ToU':
        return 1
    return 0

def count_Std(row):
    if row['stdorToU'] == 'Std':
        return 1
    return 0

def ToU_percentage(row):
    return row['ToU'] / (row['ToU'] + row['Std'])

In [6]:
altered_df_informations_households['ToU'] = df_informations_households.apply(count_ToU, axis=1)
altered_df_informations_households['Std'] = df_informations_households.apply(count_Std, axis=1)

In [ ]:
altered_df_informations_households.head()

In [8]:
counter_type_stats = altered_df_informations_households.groupby(by = 'Acorn')['ToU', 'Std'].sum()
counter_type_stats['ToU_perc'] = counter_type_stats.apply(ToU_percentage, axis=1)

### Процент счетчиков ToU в зависимости от класса

In [9]:
counter_type_stats['ToU_perc']

Acorn
ACORN-     1.000000
ACORN-A    0.248408
ACORN-B    0.160000
ACORN-C    0.231788
ACORN-D    0.250000
ACORN-E    0.216337
ACORN-F    0.223684
ACORN-G    0.229268
ACORN-H    0.184615
ACORN-I    0.156863
ACORN-J    0.276786
ACORN-K    0.133333
ACORN-L    0.222222
ACORN-M    0.150442
ACORN-N    0.171053
ACORN-O    0.155340
ACORN-P    0.200000
ACORN-Q    0.143201
ACORN-U    0.204082
Name: ToU_perc, dtype: float64

In [ ]:
altered_df_informations_households.groupby(by = 'Acorn')['ToU', 'Std'].sum()

### Расчет среднего потребления энергии в зависимости от класса

In [11]:
daily_dataset_with_hh_info = df_daily_dataset.merge(df_informations_households, how = 'left', on = 'LCLid')

In [ ]:
# daily_dataset_with_hh_info[ daily_dataset_with_hh_info['energy_count'] == 48].head()

Просмотр минимальных и максимальных дат по каждому классу

In [ ]:
# daily_dataset_with_hh_info[ daily_dataset_with_hh_info['energy_count'] == 48][['Acorn', 'day']]\
# .groupby(by = 'Acorn')['day'].agg([min, max])

#### Предварительный результат:
нужно чтобы совпадал диапазон дат для каждого класса
и нужно как то посмотреть не пропущены ли показания счетчиков в промежутке в этом диапазоне

In [23]:
counter_type_consumption = daily_dataset_with_hh_info[
    (daily_dataset_with_hh_info['energy_count'] == 48) &
    (daily_dataset_with_hh_info['Acorn'] != 'ACORN-')
].groupby(by = ['Acorn', 'stdorToU'])['energy_sum', 'energy_max'].mean()
counter_type_consumption['energy_sum']

energy_sum  energy_max
Acorn   stdorToU                        
ACORN-A Std        20.176322    1.310320
        ToU        15.983221    1.079479
ACORN-B Std        11.362745    0.862655
        ToU        14.816028    1.055368
ACORN-C Std        12.466021    0.969860
        ToU        10.360740    0.937053
ACORN-D Std        13.898305    1.004958
        ToU        12.837010    0.980037
ACORN-E Std        10.668145    0.876887
        ToU         9.392043    0.800718
ACORN-F Std         9.371257    0.756990
        ToU         8.711803    0.716672
ACORN-G Std        10.555980    0.908733
        ToU         9.019109    0.780881
ACORN-H Std        11.227942    0.893085
        ToU        10.225540    0.811497
ACORN-I Std         9.429555    0.798303
        ToU         9.713887    0.834904
ACORN-J Std        12.035216    1.152796
        ToU         9.666608    0.852311
ACORN-K Std        10.047962    0.832264
        ToU        10.130785    0.770944
ACORN-L Std        10.330252    0.840907
        ToU         9.132298    0.839893
ACORN-M Std         9.903523    0.850123
        ToU        10.828620    0.924430
ACORN-N Std         9.194234    0.788204
        ToU         9.579152    0.779315
ACORN-O Std         8.544080    0.770820
        ToU         8.766600    0.834252
ACORN-P Std         7.321499    0.851204
        ToU         3.849519    0.411216
ACORN-Q Std         7.651949    0.681974
        ToU         7.277686    0.623380
ACORN-U Std        11.877796    0.906375
        ToU        10.913723    0.975402

#### Тут видим что в среднем, там где установлен многотарифный стчетчик, - потребление энергии меньше

In [61]:
daily_dataset_with_hh_info[
    (daily_dataset_with_hh_info['energy_count'] == 48) &
    (daily_dataset_with_hh_info['Acorn'] != 'ACORN-')
].groupby(by = 'stdorToU')['energy_sum', 'energy_max'].mean()

,energy_sum,energy_max
stdorToU,,
Std,10.323564,0.847467
ToU,9.531409,0.794422


In [58]:
# def counter_consumption(row):
#     print(row[])
#     return row['Acorn']

In [59]:
acorns = pd.DataFrame({'Acorn': 'ACORN-A ACORN-B ACORN-C ACORN-D ACORN-E ACORN-F ACORN-G ACORN-H ACORN-I ACORN-J ACORN-K ACORN-L ACORN-M ACORN-N ACORN-O ACORN-P ACORN-Q ACORN-U'.split()})
#acorns['ToU'] = counter_type_consumption.apply(counter_consumption)
#acorns['ToU'] = acorns.apply(counter_consumption)